In [1]:
import numpy as np
import pandas as pd

In [2]:
data_preprocessed = pd.read_csv("Absenteeism_preprocessed.csv")
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of the week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [3]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] >
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [4]:
data_preprocessed['Excessive Absenteeism'] = targets

In [5]:
targets.sum()/targets.shape[0]

0.45571428571428574

In [6]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Day of the week', 
                                            'Distance to Work', 'Daily Work Load Average'],axis=1)

In [7]:
data_with_targets.shape

(700, 12)

In [8]:
unscaled_inputs = data_with_targets.iloc[:, :-1]

In [9]:
order = unscaled_inputs.columns.values
order

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_test_split(unscaled_inputs,targets)

[     Reason_1  Reason_2  Reason_3  Reason_4  Month  Transportation Expense  \
 456         0         0         1         0      6                     179   
 285         0         0         0         0      9                     235   
 634         1         0         0         0      3                     179   
 313         0         0         0         0     10                     157   
 362         0         0         0         1      1                     155   
 ..        ...       ...       ...       ...    ...                     ...   
 20          1         0         0         0      8                     330   
 117         0         0         0         1      1                     225   
 397         0         0         0         1      3                     378   
 25          0         0         0         1      8                     361   
 423         0         0         0         1      4                     179   
 
      Age  Body Mass Index  Education  Children  P

In [12]:
x_train,x_test,y_train,y_test = train_test_split(unscaled_inputs,targets, train_size = 0.8, random_state = 50)

In [13]:
print(x_train.shape, y_train.shape)

(560, 11) (560,)


In [14]:
print(x_test.shape, y_test.shape)

(140, 11) (140,)


In [15]:
x_test

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
16,0,0,0,1,7,179,38,31,0,0,0
454,0,0,0,1,6,118,50,31,0,1,0
77,0,0,0,1,10,289,33,30,0,2,1
282,0,0,0,1,9,260,36,23,0,4,0
531,1,0,0,0,10,225,28,24,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...
524,1,0,0,0,10,225,28,24,0,1,2
437,0,0,0,1,5,179,30,19,1,0,0
175,1,0,0,0,3,225,28,24,0,1,2
429,0,0,0,1,5,179,38,31,0,0,0


In [16]:
new_unscaled_inputs = x_train.loc[:,"Month":"Body Mass Index"]
new_unscaled_inputs_2 = x_train.loc[:,"Children":"Pets"]
dummy_1 = x_train.loc[:,"Reason_1":"Reason_4"]
dummy_2 = x_train.loc[:,"Education"]

In [17]:
new_unscaled_var = pd.concat([new_unscaled_inputs,new_unscaled_inputs_2],axis=1)
new_unscaled_var.isnull().sum()

Month                     0
Transportation Expense    0
Age                       0
Body Mass Index           0
Children                  0
Pets                      0
dtype: int64

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [19]:
scaled_new = scaler.fit_transform(new_unscaled_var)
scaled_new
scaled_df = pd.DataFrame(scaled_new, columns = ['Month', 'Transportation Expense',
                                                'Age', 'Body Mass Index',
                                                'Children', 'Pets'])
scaled_df.isnull().sum()

Month                     0
Transportation Expense    0
Age                       0
Body Mass Index           0
Children                  0
Pets                      0
dtype: int64

In [20]:
dummy_df = pd.concat([dummy_1, dummy_2],axis=1)
dummy_df.isnull().sum()

Reason_1     0
Reason_2     0
Reason_3     0
Reason_4     0
Education    0
dtype: int64

In [21]:
dummy_df.shape

(560, 5)

In [22]:
scaled_df.reset_index(drop = True)  # had to reset index because the index of "scaled_df" was randomly arranged
                                    # as we picked the x_train values, and beacuse of the improper index, it gave an error
                                    # when i tried to concat "scaled_df" and "dummy_df"

,Month,Transportation Expense,Age,Body Mass Index,Children,Pets
0,0.454628,-0.996453,-1.131382,-1.084160,-0.925174,-0.595121
1,-1.261719,-0.666968,-0.977228,-1.771995,-0.925174,-0.595121
2,0.740685,0.171723,1.026777,2.584296,-0.016231,-0.595121
3,1.598859,0.366419,1.643394,1.208625,0.892711,0.223719
4,-0.403546,-1.026407,-0.360611,-0.396324,0.892711,-0.595121
...,...,...,...,...,...,...
555,-1.547777,-0.592085,-1.439690,-1.313438,-0.925174,-0.595121
556,1.026743,0.980461,-0.514765,0.750068,0.892711,0.223719
557,1.598859,2.058777,-1.285536,0.062233,-0.016231,2.680240
558,0.454628,-0.666968,0.256006,0.979347,-0.925174,-0.595121


In [23]:
dummy_df.reset_index(drop = True)   

,Reason_1,Reason_2,Reason_3,Reason_4,Education
0,0,0,1,0,0
1,0,0,0,1,1
2,0,0,0,0,0
3,0,0,0,1,0
4,0,0,0,1,0
...,...,...,...,...,...
555,0,0,0,1,0
556,0,0,0,1,0
557,0,0,0,1,0
558,0,0,0,1,0


In [24]:
scaled_inputs = pd.concat([scaled_df, dummy_df],axis = 1)
scaled_inputs.isnull().sum()

Month                     111
Transportation Expense    111
Age                       111
Body Mass Index           111
Children                  111
Pets                      111
Reason_1                  111
Reason_2                  111
Reason_3                  111
Reason_4                  111
Education                 111
dtype: int64

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
reg = LogisticRegression(max_iter = 1000,dual = False)

In [27]:
reg.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [28]:
reg.score(x_train,y_train)

0.7625

In [29]:
reg.coef_

array([[ 2.75365334,  1.1854625 ,  3.12207949,  0.85312179,  0.01959719,
         0.00846862, -0.01619637,  0.05386594,  0.26206096,  0.34816237,
        -0.25275689]])

In [30]:
reg.intercept_

array([-4.74551844])

In [31]:
feature_names = scaled_inputs.columns.values
summary_table = pd.DataFrame(columns = ['Feature names'], data = feature_names)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ["Intercept", reg.intercept_[0]]    # Here we use intercept_[0] so that we can extract the float rather
summary_table = summary_table.sort_index()                  # than the whole array
summary_table['Odds_ratio'] = np.exp(summary_table['Coefficients'])
summary_table = summary_table.sort_values("Odds_ratio",ascending = False)
summary_table

,Feature names,Coefficients,Odds_ratio
3,Age,3.122079,22.693522
1,Month,2.753653,15.699884
2,Transportation Expense,1.185462,3.272200
4,Body Mass Index,0.853122,2.346962
10,Reason_4,0.348162,1.416462
9,Reason_3,0.262061,1.299606
8,Reason_2,0.053866,1.055343
5,Children,0.019597,1.019790
6,Pets,0.008469,1.008505
7,Reason_1,-0.016196,0.983934


In [32]:
reg.score(x_train,y_train)

0.7625

In [33]:
reg.score(x_test,y_test)

0.7928571428571428

In [34]:
y_test

array([0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1])

In [35]:
type(y_test)

numpy.ndarray

In [36]:
predicted_proba = reg.predict_proba(x_test)[:,1:]
predicted_proba

array([[0.23416316],
       [0.17259762],
       [0.56858313],
       [0.63048928],
       [0.68814891],
       [0.67893111],
       [0.21373883],
       [0.23416316],
       [0.1457228 ],
       [0.40153974],
       [0.15904993],
       [0.91416715],
       [0.91029685],
       [0.25398456],
       [0.23416316],
       [0.0773263 ],
       [0.58507217],
       [0.17403535],
       [0.25219021],
       [0.2170507 ],
       [0.40941398],
       [0.91649527],
       [0.38456835],
       [0.13623277],
       [0.80170147],
       [0.75354061],
       [0.66729663],
       [0.88880755],
       [0.96150888],
       [0.25172093],
       [0.52490387],
       [0.24851235],
       [0.53466737],
       [0.1797416 ],
       [0.16510968],
       [0.6073843 ],
       [0.18559291],
       [0.80893079],
       [0.8352349 ],
       [0.21662787],
       [0.24851235],
       [0.6073843 ],
       [0.23371694],
       [0.74904002],
       [0.84688472],
       [0.25478004],
       [0.30794269],
       [0.644

In [37]:
predicted = reg.predict(x_test)
predicted

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0])

In [38]:
x_test = x_test.assign(Probabilities = predicted_proba)
x_test

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Probabilities
16,0,0,0,1,7,179,38,31,0,0,0,0.234163
454,0,0,0,1,6,118,50,31,0,1,0,0.172598
77,0,0,0,1,10,289,33,30,0,2,1,0.568583
282,0,0,0,1,9,260,36,23,0,4,0,0.630489
531,1,0,0,0,10,225,28,24,0,1,2,0.688149
...,...,...,...,...,...,...,...,...,...,...,...,...
524,1,0,0,0,10,225,28,24,0,1,2,0.688149
437,0,0,0,1,5,179,30,19,1,0,0,0.185593
175,1,0,0,0,3,225,28,24,0,1,2,0.657979
429,0,0,0,1,5,179,38,31,0,0,0,0.227208


In [39]:
x_test = x_test.assign(Predicted_Absenteeism = predicted)
x_test

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Probabilities,Predicted_Absenteeism
16,0,0,0,1,7,179,38,31,0,0,0,0.234163,0
454,0,0,0,1,6,118,50,31,0,1,0,0.172598,0
77,0,0,0,1,10,289,33,30,0,2,1,0.568583,1
282,0,0,0,1,9,260,36,23,0,4,0,0.630489,1
531,1,0,0,0,10,225,28,24,0,1,2,0.688149,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,1,0,0,0,10,225,28,24,0,1,2,0.688149,1
437,0,0,0,1,5,179,30,19,1,0,0,0.185593,0
175,1,0,0,0,3,225,28,24,0,1,2,0.657979,1
429,0,0,0,1,5,179,38,31,0,0,0,0.227208,0


In [40]:
x_test

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Probabilities,Predicted_Absenteeism
16,0,0,0,1,7,179,38,31,0,0,0,0.234163,0
454,0,0,0,1,6,118,50,31,0,1,0,0.172598,0
77,0,0,0,1,10,289,33,30,0,2,1,0.568583,1
282,0,0,0,1,9,260,36,23,0,4,0,0.630489,1
531,1,0,0,0,10,225,28,24,0,1,2,0.688149,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,1,0,0,0,10,225,28,24,0,1,2,0.688149,1
437,0,0,0,1,5,179,30,19,1,0,0,0.185593,0
175,1,0,0,0,3,225,28,24,0,1,2,0.657979,1
429,0,0,0,1,5,179,38,31,0,0,0,0.227208,0


In [41]:
x_test.to_csv("Absenteeism Predictions.csv",index = False)